<a href="https://colab.research.google.com/github/FatmaBuseBorlu/Yapay-Zeka-DREAMER/blob/main/gemini_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==============================================================================
# EEG İLE DUYGU TANIMA PROJESİ - HATA GİDERİLMİŞ VE OPTİMİZE EDİLMİŞ KOD (v4.2)
# ==============================================================================

# ------------------------------------------------------------------------------
# Gerekli Kütüphaneler
# ------------------------------------------------------------------------------
import numpy as np
import scipy.io
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import time
import os

# Google Drive'ı bağla (Colab ortamında gerekli)
from google.colab import drive
drive.mount('/content/drive')

# ------------------------------------------------------------------------------
# Yapılandırma ve Hata Çözümü Yolları
# ------------------------------------------------------------------------------
DRIVE_FILEPATH = '/content/drive/MyDrive/Colab Notebooks/EEG Tabanlı Duygu Tanımada Hibrit CNN-LSTM Modellerinin Başarımı: DREAMER Veri Seti Üzerine Bir Analiz/DREAMER.mat'
LOCAL_FILEPATH = '/tmp/DREAMER.mat'  # Dosyanın kopyalanacağı yerel yol

TARGET_EMOTION = 'Arousal'
EPOCHS = 100
BATCH_SIZE = 256
SEGMENT_LENGTH = 256
DROPOUT_RATE = 0.7
PATIENCE = 10

# ------------------------------------------------------------------------------
# 1. VERİ YÜKLEME VE İŞLEME FONKSİYONLARI
# ------------------------------------------------------------------------------
def load_and_process_dreamer_data(filepath):
    """
    Standart DREAMER.mat dosyasını yükler, baseline düzeltmesi yapar.
    """
    if not os.path.exists(filepath):
        print(f"HATA: Dosya bulunamadı: {filepath}")
        return None, None, None

    # MATLAB dosyasını yerel yoldan okuma
    try:
        mat = scipy.io.loadmat(filepath)
    except Exception as e:
        print(f"MATLAB dosyası okunurken hata oluştu: {e}")
        return None, None, None

    all_eeg_data, all_labels_valence, all_labels_arousal = [], [], []

    try:
        data_struct = mat['DREAMER'][0, 0]['Data'][0]
    except KeyError:
        print("HATA: Dosya yapısı beklenenden farklı.")
        return None, None, None

    for subject_id in range(len(data_struct)):
        subject = data_struct[subject_id]
        subject_eeg_list, subject_labels_valence, subject_labels_arousal = [], [], []

        for video_id in range(18):
            try:
                stimuli_signal = subject['EEG'][0,0]['stimuli'][0,0][video_id,0]
                baseline_signal = subject['EEG'][0,0]['baseline'][0,0][video_id,0]
                valence_val = subject['ScoreValence'][0,0][video_id,0]
                arousal_val = subject['ScoreArousal'][0,0][video_id,0]

            except (IndexError, KeyError):
                continue

            if stimuli_signal.ndim == 2 and baseline_signal.ndim == 2:
                # Baseline çıkarma
                baseline_mean = np.mean(baseline_signal, axis=0)
                eeg_baselined = stimuli_signal - baseline_mean

                subject_eeg_list.append(eeg_baselined)
                subject_labels_valence.append(int(valence_val))
                subject_labels_arousal.append(int(arousal_val))

        all_eeg_data.append(subject_eeg_list)
        all_labels_valence.append(subject_labels_valence)
        all_labels_arousal.append(subject_labels_arousal)

    return all_eeg_data, all_labels_valence, all_labels_arousal

def create_segments(eeg_signals, labels, window_size):
    """
    EEG sinyallerini segmentlere ayırır ve nötr etiketli (3 puanlı) klibin
    tüm segmentlerini atlayarak ikili (binary) sınıflamaya dönüştürür.

    Hata Düzeltme Notu: Segmentler ve etiketler, sadece etiket != 3 olduğunda
    aynı anda eklenir, böylece uzunluk tutarsızlığı (ValueError) önlenir.
    """
    segments, segment_labels = [], []
    overlap = window_size // 2

    for i in range(len(eeg_signals)):
        signal, label = eeg_signals[i], labels[i]

        # Sadece nötr olmayan etiketler (1, 2, 4, 5) için segmentasyon yap
        if label != 3:
            # İkili Sınıflama etiketini belirle: 1-2 -> 0, 4-5 -> 1
            binary_label = 1 if label > 3 else 0

            # Segmentasyon döngüsü
            for start in range(0, signal.shape[0] - window_size, window_size - overlap):
                # Segment ve etiketi aynı anda, koşullu olarak ekle
                segments.append(signal[start : start + window_size, :])
                segment_labels.append(binary_label)

    return np.array(segments), np.array(segment_labels)

# ------------------------------------------------------------------------------
# 2. İYİLEŞTİRİLMİŞ MODEL TANIMLAMA FONKSİYONLARI
# ------------------------------------------------------------------------------
def build_cnn_model(input_shape, num_classes):
    """Sadece CNN Modeli (Dropout ve Kernel boyutu iyileştirildi)"""
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=7, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(DROPOUT_RATE)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_lstm_model(input_shape, num_classes):
    """Sadece LSTM Modeli (Bidirectional ve Dropout iyileştirildi)"""
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(64, return_sequences=True))(inputs) # Bidirectional eklendi
    x = Bidirectional(LSTM(32))(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(DROPOUT_RATE)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_lstm_model(input_shape, num_classes):
    """Hibrit CNN-LSTM Modeli (Bidirectional ve Dropout iyileştirildi)"""
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=7, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Bidirectional(LSTM(64))(x) # Bidirectional eklendi
    x = Dense(100, activation='relu')(x)
    x = Dropout(DROPOUT_RATE)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ------------------------------------------------------------------------------
# 3. ANA EĞİTİM VE DEĞERLENDİRME KODU
# ------------------------------------------------------------------------------

# Drive Kopyalama Adımı (OSError çözümü)
print("DREAMER standart veri seti yükleniyor...")
if os.path.exists(DRIVE_FILEPATH):
    print(f"Drive dosyasını ({DRIVE_FILEPATH}) yerel dizine kopyalanıyor...")
    !cp "{DRIVE_FILEPATH}" "{LOCAL_FILEPATH}"
    if os.path.exists(LOCAL_FILEPATH):
        print("✅ Kopyalama başarılı. Okuma işlemi yerel dizinden devam edecek.")
    else:
        print("HATA: Kopyalama başarısız oldu. Orijinal yoldan deneniyor.")
        LOCAL_FILEPATH = DRIVE_FILEPATH
else:
    print(f"HATA: Drive dosya yolu bulunamadı: {DRIVE_FILEPATH}. Lütfen yolu kontrol edin.")

# Veri yüklemesi kopyalanan yerel yoldan yapılır
eeg_signals, valence_labels, arousal_labels = load_and_process_dreamer_data(LOCAL_FILEPATH)

if eeg_signals and eeg_signals[0]: # Veri başarılıysa devam et
    print("✅ Veri yüklemesi başarıyla tamamlandı.")
    target_labels = arousal_labels if TARGET_EMOTION == 'Arousal' else valence_labels
    print(f"Hedef duygu olarak '{TARGET_EMOTION}' seçildi.")

    all_scores = { "CNN": [], "LSTM": [], "CNN-LSTM": [] }
    loo = LeaveOneOut()
    subjects = list(range(len(eeg_signals)))

    # Erken Durdurma Callback'i tanımlanıyor
    early_stopping = EarlyStopping(
        monitor='loss',
        patience=PATIENCE,
        restore_best_weights=True
    )

    for fold, (train_indices, test_indices) in enumerate(loo.split(subjects)):
        test_subject_id = test_indices[0]
        print(f"\n===== FOLD {fold+1}/{len(subjects)} | Test Katılımcısı: {test_subject_id+1} =====")

        X_train_list, y_train_list = [], []
        for sub_idx in train_indices:
            segments, labels = create_segments(eeg_signals[sub_idx], target_labels[sub_idx], SEGMENT_LENGTH)
            if len(segments) > 0:
                X_train_list.append(segments)
                y_train_list.append(labels)

        X_test, y_test = create_segments(eeg_signals[test_subject_id], target_labels[test_subject_id], SEGMENT_LENGTH)

        if not X_train_list or len(X_test) == 0:
            print("Eğitim veya test için yeterli veri yok, bu fold atlanıyor.")
            continue

        X_train = np.concatenate(X_train_list, axis=0)
        y_train = np.concatenate(y_train_list, axis=0)

        # Sınıf ağırlıklarını hesaplama (Dengesizlik giderme)
        class_weights_array = class_weight.compute_class_weight(
            'balanced',
            classes=np.unique(y_train),
            y=y_train
        )
        class_weights = dict(zip(np.unique(y_train), class_weights_array))

        # Veri Normalizasyonu (Standartlaştırma)
        scaler = StandardScaler()
        X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
        X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

        X_train = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
        X_test = scaler.transform(X_test_reshaped).reshape(X_test.shape)

        y_train_cat = to_categorical(y_train, num_classes=2)

        input_shape = (X_train.shape[1], X_train.shape[2])
        models = {
            "CNN": build_cnn_model(input_shape, 2),
            "LSTM": build_lstm_model(input_shape, 2),
            "CNN-LSTM": build_cnn_lstm_model(input_shape, 2)
        }

        for name, model in models.items():
            print(f"\n--- {name} modeli eğitiliyor... ---")

            # Erken durdurma ve sınıf ağırlıkları kullanılarak eğitim
            model.fit(
                X_train,
                y_train_cat,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                verbose=0,
                callbacks=[early_stopping],
                class_weight=class_weights
            )

            # Test ve Sonuç Raporlama
            y_pred_probs = model.predict(X_test, verbose=0)
            y_pred = np.argmax(y_pred_probs, axis=1)

            # Test setinde sadece bir sınıf varsa F1 hesaplama hatası olabileceği için kontrol eklenmiştir
            if len(np.unique(y_test)) > 1:
                f1 = f1_score(y_test, y_pred, average='weighted')
            else:
                f1 = 0.0 # Veya sadece doğruluk kullanılır

            acc = accuracy_score(y_test, y_pred)
            all_scores[name].append({'acc': acc, 'f1': f1})
            print(f"-> {name} Modeli Sonuçları: Doğruluk={acc:.4f}, F1-Skor={f1:.4f}")

    print(f"\n\nTüm eğitim süreci tamamlandı.")
    print("\n\n===== TÜM KATILIMCILAR İÇİN ORTALAMA PERFORMANS SONUÇLARI =====")
    for name in all_scores:
        if all_scores[name]:
            avg_acc = np.mean([s['acc'] for s in all_scores[name]])
            std_acc = np.std([s['acc'] for s in all_scores[name]])
            print(f"Model: {name}\t Ortalama Doğruluk: {avg_acc:.4f} (± {std_acc:.4f})")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DREAMER standart veri seti yükleniyor...
Drive dosyasını (/content/drive/MyDrive/Colab Notebooks/EEG Tabanlı Duygu Tanımada Hibrit CNN-LSTM Modellerinin Başarımı: DREAMER Veri Seti Üzerine Bir Analiz/DREAMER.mat) yerel dizine kopyalanıyor...
✅ Kopyalama başarılı. Okuma işlemi yerel dizinden devam edecek.
✅ Veri yüklemesi başarıyla tamamlandı.
Hedef duygu olarak 'Arousal' seçildi.

===== FOLD 1/23 | Test Katılımcısı: 1 =====

--- CNN modeli eğitiliyor... ---
-> CNN Modeli Sonuçları: Doğruluk=0.5243, F1-Skor=0.4981

--- LSTM modeli eğitiliyor... ---
-> LSTM Modeli Sonuçları: Doğruluk=0.4259, F1-Skor=0.4004

--- CNN-LSTM modeli eğitiliyor... ---
-> CNN-LSTM Modeli Sonuçları: Doğruluk=0.4730, F1-Skor=0.4211

===== FOLD 2/23 | Test Katılımcısı: 2 =====

--- CNN modeli eğitiliyor... ---
-> CNN Modeli Sonuçları: Doğruluk=0.4772, F1-Skor=0.5066

--- LSTM modeli eğiti